<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Objectives" data-toc-modified-id="Objectives-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Objectives</a></span></li><li><span><a href="#Aggregating-Functions" data-toc-modified-id="Aggregating-Functions-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Aggregating Functions</a></span><ul class="toc-item"><li><span><a href="#Example-Simple-Aggregations" data-toc-modified-id="Example-Simple-Aggregations-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Example Simple Aggregations</a></span></li></ul></li><li><span><a href="#Grouping-in-SQL" data-toc-modified-id="Grouping-in-SQL-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Grouping in SQL</a></span><ul class="toc-item"><li><span><a href="#Example-GROUP-BY--Statements" data-toc-modified-id="Example-GROUP-BY--Statements-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Example <code>GROUP BY</code>  Statements</a></span><ul class="toc-item"><li><span><a href="#Without-GROUP-BY" data-toc-modified-id="Without-GROUP-BY-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Without <code>GROUP BY</code></a></span></li><li><span><a href="#With-GROUP-BY" data-toc-modified-id="With-GROUP-BY-3.1.2"><span class="toc-item-num">3.1.2&nbsp;&nbsp;</span>With <code>GROUP BY</code></a></span></li></ul></li><li><span><a href="#Group-Task" data-toc-modified-id="Group-Task-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Group Task</a></span><ul class="toc-item"><li><span><a href="#Possible-Solution" data-toc-modified-id="Possible-Solution-3.2.1"><span class="toc-item-num">3.2.1&nbsp;&nbsp;</span>Possible Solution</a></span></li></ul></li><li><span><a href="#Exercise:-Grouping" data-toc-modified-id="Exercise:-Grouping-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Exercise: Grouping</a></span><ul class="toc-item"><li><span><a href="#Possible-Solution" data-toc-modified-id="Possible-Solution-3.3.1"><span class="toc-item-num">3.3.1&nbsp;&nbsp;</span>Possible Solution</a></span></li></ul></li></ul></li><li><span><a href="#Filtering-Groups-with-HAVING" data-toc-modified-id="Filtering-Groups-with-HAVING-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Filtering Groups with <code>HAVING</code></a></span><ul class="toc-item"><li><span><a href="#Examples-of-Using-HAVING" data-toc-modified-id="Examples-of-Using-HAVING-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Examples of Using <code>HAVING</code></a></span><ul class="toc-item"><li><span><a href="#Simple-Filtering---Number-of-Airports-in-a-Country" data-toc-modified-id="Simple-Filtering---Number-of-Airports-in-a-Country-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>Simple Filtering - Number of Airports in a Country</a></span></li></ul></li><li><span><a href="#Filtering-Different-Aggregation---Airport-Altitudes" data-toc-modified-id="Filtering-Different-Aggregation---Airport-Altitudes-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Filtering Different Aggregation - Airport Altitudes</a></span><ul class="toc-item"><li><span><a href="#Looking-at-the-airports-Table" data-toc-modified-id="Looking-at-the-airports-Table-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Looking at the <code>airports</code> Table</a></span></li><li><span><a href="#Looking-at-the-Highest-Airport" data-toc-modified-id="Looking-at-the-Highest-Airport-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Looking at the Highest Airport</a></span></li><li><span><a href="#Looking-at-the-Number-of-Airports-Too" data-toc-modified-id="Looking-at-the-Number-of-Airports-Too-4.2.3"><span class="toc-item-num">4.2.3&nbsp;&nbsp;</span>Looking at the Number of Airports Too</a></span></li><li><span><a href="#Finally-Filter-Aggregation" data-toc-modified-id="Finally-Filter-Aggregation-4.2.4"><span class="toc-item-num">4.2.4&nbsp;&nbsp;</span>Finally Filter Aggregation</a></span></li></ul></li></ul></li></div>

![sql](img/sql-logo.jpg)

In [2]:
# Import data cleaning/processing library
import pandas as pd

# Import file-based database library 
import sqlite3

# open connection to the flights database.
conn = sqlite3.connect("flights.db")

# create cursor object which will allow us to interact with the flights database.
cur = conn.cursor()

# Objectives

- Use SQL aggregation functions with GROUP BY
- Use HAVING for group filtering
- Use SQL JOIN to combine tables using keys

# Recap
lets reacquaint ourselves with our db using steps from last lecture:

### Explore the schema of our database -> 

**database schema**: a master listing of all database objects in the database and the SQL used to create each object.

**tables**: data that is logically organized in a row-and-column format similar to a spreadsheet. Each row represents a unique record, and each column represents a field in the record.

**index**: special lookup tables that the database search engine can use to speed up data retrieval.

In [3]:
schema_df = pd.read_sql("""

SELECT *
FROM sqlite_master

""", conn)

schema_df

,type,name,tbl_name,rootpage,sql
0,table,airports,airports,2,"CREATE TABLE airports (\n[index] INTEGER,\n [..."
1,index,ix_airports_index,airports,3,CREATE INDEX ix_airports_index ON airports ([i...
2,table,airlines,airlines,945,"CREATE TABLE airlines (\n[index] INTEGER,\n [..."
3,index,ix_airlines_index,airlines,946,CREATE INDEX ix_airlines_index ON airlines ([i...
4,table,routes,routes,1393,"CREATE TABLE routes (\n[index] INTEGER,\n [ai..."
5,index,ix_routes_index,routes,1394,CREATE INDEX ix_routes_index ON routes ([index])


In [ ]:
schema_df.iloc[0]

In [ ]:
airports_desc = schema_df.iloc[0]['sql']

In [ ]:
# We can also do get the same information by reading a SQL query for the airports table 
# into a pandas dataframe and inspect using .info()

pd.read_sql('''
    SELECT *
    FROM airports
''',conn).info()

# Aggregating Functions

>  A SQL **aggregating function** takes in many values and returns one value.

We have already seen some SQL aggregating functions like `COUNT()`. There are also others, like SUM(), AVG(), MIN(), and MAX().

## Example Simple Aggregations

In [ ]:
# Max value for longitude (comments in sql are indicated with a '--')
pd.read_sql('''
    SELECT 
        -- Note we have to cast to a numerical value first
        MAX(
            CAST(longitude AS REAL)
        ) AS "max_longitude"
    FROM 
        airports
''', conn)

In [ ]:
# Max value for id in table
pd.read_sql('''
SELECT 
    MAX(CAST(id AS integer)) AS "max_id"
FROM 
    airports
''', conn)

In [ ]:
# Effectively counts all the inactive airlines 
pd.read_sql('''
    SELECT 
        COUNT() AS "num_airlines_inactive"
    FROM 
        airlines
    WHERE 
        active='N'
''', conn)

We can also give aliases to our aggregations:

In [ ]:
# Effectively counts all the active airlines 
pd.read_sql('''
    SELECT 
        COUNT() AS number_of_active_airlines
    FROM 
        airlines
    WHERE 
        active='Y'
''', conn)

# Grouping in SQL

We can go deeper and use aggregation functions on _groups_ using the `GROUP BY` clause.

The `GROUP BY` clause will group one or more columns together with the same values as one group to perform aggregation functions on much like the `.groupby()` operation/method in pandas.

## Example `GROUP BY`  Statements

Let's say we want to know how many active and non-active airlines there are.

### Without `GROUP BY`

Let's first start with just seeing how many airlines there are:

In [ ]:
df_results = pd.read_sql('''
    SELECT 
        -- Reminder that this counts the number of rows of both active/inactive airlines
        COUNT() AS number_of_airlines
    FROM 
        airlines
''', conn)

df_results

One way for us to get the counts for each is to create two queries that will filter each kind of airline (active vs non-active) and count those values:

In [ ]:
df_active = pd.read_sql('''
    SELECT 
        COUNT() AS number_of_active_airlines
    FROM 
        airlines
    WHERE 
        active='Y'
''', conn)

df_not_active = pd.read_sql('''
    SELECT 
        COUNT() AS number_of_inactive_airlines
    FROM 
        airlines
    WHERE 
        active='N'
''', conn)

display(df_active)
display(df_not_active)

This works but it's inefficient.

### With `GROUP BY`

Instead, we can tell the SQL server to do the work for us by grouping values we care about for us!

In [ ]:
df_results = pd.read_sql('''
    SELECT 
        COUNT() AS number_of_airlines
    FROM 
        airlines
    GROUP BY
        active
''', conn)

df_results

This is great! And if you look closely, you can observe we have _three_ different groups instead of our expected two!

Because active was not referenced in our select statement, the query will return the information we want without
reference to the categories we're interested in. 

Let's also print out the `airlines.active` value for each group/aggregation so we know what we're looking at:

In [4]:
df_results = pd.read_sql('''
    SELECT 
        
        airlines.active,
        COUNT() AS number_of_airlines
    FROM 
        airlines
    GROUP BY
        airlines.active
''', conn)

df_results

,active,number_of_airlines
0,N,4886
1,Y,1161
2,n,1


## Group Task

- Which countries have the highest numbers of active airlines? Return the top 10.

In [ ]:
pd.read_sql('''
    SELECT COUNT 
        *
    FROM
        airlines
''', conn).head()

<details>
    <summary><b>Possible Solution</b></summary>

``` sql
    pd.read_sql('''
    SELECT 
        COUNT() AS num,
        country
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    ORDER BY 
        num DESC
    LIMIT 10
    ''', conn)```
</details>

> Note that the `GROUP BY` clause is considered _before_ the `ORDER BY` and `LIMIT` clauses

## Exercise: Grouping

- Run a query that will return the number of airports by time zone. Each row should have a number of airports and a time zone.

In [9]:
# Your code here
pd.read_sql('''
 SELECT 
        airports.timezone
        ,COUNT() AS num_of_airports
    FROM 
        airports
    GROUP BY
        airports.timezone
    ORDER BY
        num_of_airports DESC

''', conn).head()

,timezone,num_of_airports
0,America/New_York,628
1,America/Chicago,373
2,Europe/Berlin,319
3,America/Anchorage,258
4,Europe/Paris,232


<details>
    <summary><b>Possible Solution</b></summary>

``` sql
pd.read_sql('''
    SELECT 
        airports.timezone
        ,COUNT() AS num_of_airports
    FROM 
        airports
    GROUP BY
        airports.timezone
    ORDER BY
        num_of_airports DESC
''', conn) ```
</details>

# Filtering Groups with `HAVING`

We showed that you can filter tables with `WHERE`. We can similarly filter _groups/aggregations_ using `HAVING` clauses.

## Examples of Using `HAVING`

### Simple Filtering - Number of Airports in a Country

Let's come back to the aggregation of active airports:

In [ ]:
pd.read_sql('''
    SELECT 
        COUNT() AS num,
        country
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    ORDER BY 
        num DESC
''', conn)

We can see we have a lot of results. But maybe we only want to keep the countries that have more than $30$ active airlines:

In [ ]:
pd.read_sql('''
    SELECT 
        country,
        COUNT() AS num
    FROM 
        airlines
    WHERE 
        active='Y'
    GROUP BY 
        country
    HAVING
        num > 30
    ORDER BY 
        num DESC
''', conn)

## Filtering Different Aggregations - Airport Altitudes

We can also filter on other aggregations. For example, let's say we want to investigate the `airports` table.

Specifically, we want to know the height of the _highest airport_ in a country given that it has _at least $100$ airports_.

### Looking at the `airports` Table

In [ ]:
df_airports = pd.read_sql('''
    SELECT 
        *
    FROM 
        airports 
''', conn)

df_airports.info()

In [ ]:
df_airports.head()

### Looking at the Highest Airport

Let's first get the highest altitude for each airport:

In [ ]:
pd.read_sql('''
    SELECT 
        airports.country
        ,MAX(
            CAST(airports.altitude AS REAL)   -- REAL gives less decimal points than float. 
        ) AS highest_airport_in_country
    FROM 
        airports 
    GROUP BY
        airports.country
    ORDER BY
        airports.country   -- ORDER BY for string alphabetizes.
''', conn)

### Looking at the Number of Airports Too

We can also get the number of airports for each country.

In [ ]:
pd.read_sql('''
    SELECT 
        airports.country
        ,MAX(
            CAST(airports.altitude AS REAL)
        ) AS highest_airport_in_country
        ,COUNT() AS number_of_airports_in_country
    FROM
        airports 
    GROUP BY
        airports.country
    ORDER BY
        airports.country
''', conn)

### Filtering on Aggregations

> Recall:
>
> We want to know the height of the _highest airport_ in a country given that it has _at least $100$ airports_.

In [ ]:
pd.read_sql('''
    SELECT 
        airports.country
        ,MAX(
            CAST(airports.altitude AS REAL)
        ) AS highest_airport_in_country
        -- Note we don't have to include this in our SELECT
        ,COUNT() AS number_of_airports_in_country
    FROM
        airports 
    GROUP BY
        airports.country
    HAVING
        number_of_airports_in_country >= 100
    ORDER BY
        airports.country
''', conn)